# Imports

In [1]:
import gurobipy as gp
from gurobipy import GRB
import random

## Create Model

In [2]:
model = gp.Model("Two-Stage Stochastic Programming Model for Vessel chartering strategies for Offshore Wind Farms")

Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-29


# First-Stage Model

In [3]:
# Sets
V = ["Crew Transfer Vessel", "Service Operation Vessel"]
T = [
    "January"
    #"February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"
]

# Parameters
C_ST = {(v, t): 50 if v == "Crew Transfer Vessel" else 150 for v in V for t in T}

C_LT = {
    "Crew Transfer Vessel": 500,
    "Service Operation Vessel": 2_800,
}

# Variables
gamma_ST = model.addVars(V, T, vtype=GRB.INTEGER, name="gamma_ST")
gamma_LT = model.addVars(V, vtype=GRB.INTEGER, name="gamma_LT")

# Second-Stage Model

## Second-Stage Sets, Parameters and Variables

In [4]:
random.seed(18)

# Sets
W = ["Wind Farm A", "Wind Farm B"]
M = ["Small Maintenance", "Large Maintenance"]
K_v = {
    "Crew Transfer Vessel": [0],
    "Service Operation Vessel": [0, 1, 2]
}
D = [d for d in range(len(T) * 30)]
D_t = {month: D[i * 30 : (i + 1) * 30] for i, month in enumerate(T)}

S = range(1) # scenarios

# Parameters

P_mk = { 
    "Small Maintenance": [4, 2, 0],
    "Large Maintenance": [0, 1, 2]
}
L_k = {
    0: 4,
    1: 5,
    2: 4
}
L_vw = {
    ("Crew Transfer Vessel", "Wind Farm A"): 2,
    ("Crew Transfer Vessel", "Wind Farm B"): 3,
    ("Service Operation Vessel", "Wind Farm A"): 4,
    ("Service Operation Vessel", "Wind Farm B"): 5
}

# Terskel for minimum værvindu (f.eks. 3 timer)
min_window = 5

#duration (in hours) of longest weather window in scenario s on day d, 0 if too short
A_vwd_s = {} # (v, w, d, s) -> duration of longest weather window in hours

for v in V:
    for w in W:
        for d in D:
            for s in S:
                duration = random.randint(0, 12)  # Simulerer værvindu mellom 0 og 12 timer
                A_vwd_s[(v, w, d, s)] = duration if duration >= min_window else 0


N_v = {
    "Crew Transfer Vessel": 2,
    "Service Operation Vessel": 14
}

"""
C_U = {
    ("Crew Transfer Vessel", "Wind Farm A"): 1,
    ("Crew Transfer Vessel", "Wind Farm B"): 1,
    ("Service Operation Vessel", "Wind Farm A"): 2,
    ("Service Operation Vessel", "Wind Farm B"): 3
}
"""

B_mw_0 = 0
# number of failures of maintenance tasks of category m that occurs at wind farm w in scenario s on day d
F_mwd_s = {}

for m in M:
    for w in W:
        for d in D:
            for s in S:
                if m == "Small Maintenance":
                    F_mwd_s[(m, w, d, s)] = random.randint(0, 10)  # Simulerer 0-2 små vedlikeholdsoppgaver
                else:
                    F_mwd_s[(m, w, d, s)] = random.randint(0, 2)  # Simulerer 0-1 store vedlikeholdsoppgaver

print(F_mwd_s)
# Variables
x = model.addVars(V, W, D, S, vtype=GRB.INTEGER, name="x")
l = model.addVars(
    ((v, w, d, k, s) 
    for v in V
    for w in W
    for d in D
    for k in K_v[v]
    for s in S),
    vtype=GRB.INTEGER, 
    name="lambda"
)
z = model.addVars(W, M, D, S,  vtype=GRB.INTEGER, name="z")
b = model.addVars(W, M, D, S, vtype=GRB.INTEGER, name="b")

C_D = {key: 8 for key in b.keys()}
C_U = {key: 2 for key in x.keys()}

{('Small Maintenance', 'Wind Farm A', 0, 0): 1, ('Small Maintenance', 'Wind Farm A', 1, 0): 7, ('Small Maintenance', 'Wind Farm A', 2, 0): 6, ('Small Maintenance', 'Wind Farm A', 3, 0): 6, ('Small Maintenance', 'Wind Farm A', 4, 0): 2, ('Small Maintenance', 'Wind Farm A', 5, 0): 6, ('Small Maintenance', 'Wind Farm A', 6, 0): 6, ('Small Maintenance', 'Wind Farm A', 7, 0): 2, ('Small Maintenance', 'Wind Farm A', 8, 0): 4, ('Small Maintenance', 'Wind Farm A', 9, 0): 6, ('Small Maintenance', 'Wind Farm A', 10, 0): 10, ('Small Maintenance', 'Wind Farm A', 11, 0): 5, ('Small Maintenance', 'Wind Farm A', 12, 0): 2, ('Small Maintenance', 'Wind Farm A', 13, 0): 5, ('Small Maintenance', 'Wind Farm A', 14, 0): 5, ('Small Maintenance', 'Wind Farm A', 15, 0): 9, ('Small Maintenance', 'Wind Farm A', 16, 0): 4, ('Small Maintenance', 'Wind Farm A', 17, 0): 2, ('Small Maintenance', 'Wind Farm A', 18, 0): 6, ('Small Maintenance', 'Wind Farm A', 19, 0): 8, ('Small Maintenance', 'Wind Farm A', 20, 0): 0, 

## Objective Function

In [5]:
first_stage_obj = gamma_ST.prod(C_ST) + gamma_LT.prod(C_LT)

expected_second_stage_obj = (1 / len(S)) * (b.prod(C_D) + x.prod(C_U))

model.setObjective(first_stage_obj + expected_second_stage_obj)

## Second-Stage Constraints

In [6]:
# The number of vessels allocated in the second stage cannot 
# exceed the number of vessels chartered in the first stage
model.addConstrs(
    (x.sum(v, '*', d, s) <= gamma_ST[v, t] + gamma_LT[v] 
    for v in V 
    for t in T 
    for d in D_t[t] 
    for s in S),
    name="vessels_available"
)

model.addConstrs(
    l.sum(v, w, d, '*', s) <= N_v[v] * x[v, w, d, s]
    for v in V
    for w in W
    for d in D
    for s in S
)

model.addConstrs(
    (L_k[k] + L_vw[v, w] - A_vwd_s[v,w,d,s])*l[v,w,d,k,s] <=0
    for v in V
    for w in W
    for d in D
    for k in K_v[v]
    for s in S
)

#connect maintenance tasks performed to patterns chosen (9)
for w in W:
    for m in M:
        for d in D:
            for s in S:
                model.addConstr(
                    z[w, m, d, s] <= gp.quicksum(P_mk[m][k] * l[v, w, d, k, s] for v in V for k in K_v[v]),
                    name=f"Maintenance_Task_Completion_m{m}_w{w}_d{d}_s{s}"
                )

model.addConstrs(
    b[w,m,0,s] == B_mw_0 
    for w in W 
    for m in M 
    for s in S
)

model.addConstrs(
    b[w,m,d,s] == b[w,m,d-1,s] + F_mwd_s[m,w,d,s] - z[w,m,d,s]
    for w in W
    for m in M
    for d in D[1:]
    for s in S
)
model.optimize()

for v in V:
    for t in T:
        if gamma_ST[v, t].X > 0:
            print(f"Short-term charter {gamma_ST[v, t].X} vessels of type {v} in month {t}")
for v in V:
    if gamma_LT[v].X > 0:
        print(f"Long-term charter {gamma_LT[v].X} vessels of type {v}")


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 660 rows, 604 columns and 1594 nonzeros
Model fingerprint: 0xecd12b9b
Variable types: 0 continuous, 604 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 34310.000000
Presolve removed 478 rows and 371 columns
Presolve time: 0.00s
Presolved: 182 rows, 233 columns, 524 nonzeros
Found heuristic solution: objective 23010.000000
Variable types: 0 continuous, 233 integer (3 binary)

Root relaxation: objective 1.966695e+03, 111 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    